In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
path = r'C:\Users\ishit\Downloads\Image\Image'
files = os.listdir(path)
files.sort()
print(files)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']


In [6]:
image_array = []
label_array = []

In [7]:
for i in tqdm(range(len(files))):
	sub_file=os.listdir(path+"/"+files[i])

	for j in range(len(sub_file)):

		file_path=path+"/"+files[i]+"/"+sub_file[j]

		image=cv2.imread(file_path)

		image=cv2.resize(image,(96,96))

		image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)


		image_array.append(image)

		label_array.append(i)

100%|██████████| 26/26 [00:11<00:00,  2.32it/s]


In [8]:
image_array=np.array(image_array)
label_array=np.array(label_array,dtype="float")

In [9]:
from sklearn.model_selection import train_test_split
# output									   train image   label      spliting size
X_train,X_test,Y_train,Y_test=train_test_split(image_array,label_array,test_size=0.15)

del image_array,label_array

In [10]:
import gc
gc.collect()

9

In [11]:
from keras import layers,callbacks,utils,applications,optimizers
from keras.models import Sequential, Model, load_model

In [12]:
model=Sequential()

In [13]:
pretrained_model=tf.keras.applications.EfficientNetB0(input_shape=(96,96,3),include_top=False)
model.add(pretrained_model)

16705208/16705208 [==============================] - 2s 0us/step


In [14]:
model.add(layers.GlobalAveragePooling2D())

In [15]:
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1))
model.build(input_shape=(None,96,96,3))

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb0 (Functional)  (None, 3, 3, 1280)       4049571   
                                                                 
 global_average_pooling2d (G  (None, 1280)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dropout (Dropout)           (None, 1280)              0         
                                                                 
 dense (Dense)               (None, 1)                 1281      
                                                                 
Total params: 4,050,852
Trainable params: 4,008,829
Non-trainable params: 42,023
_________________________________________________________________


In [17]:
model.compile(optimizer="adam",loss="mae",metrics=["mae"])

In [18]:
ckp_path="trained_model/model"
model_checkpoint=tf.keras.callbacks.ModelCheckpoint(
									filepath=ckp_path,
									monitor="val_mae",
									mode="auto",
									save_best_only=True,
									save_weights_only=True
									)

In [19]:
reduce_lr=tf.keras.callbacks.ReduceLROnPlateau(
									factor=0.9,
									monitor="val_mae",
									mode="auto",
									cooldown=0,
									patience=5,
									verbose=1,
									min_lr=1e-6)

In [20]:
Epochs=100
Batch_Size=32

In [21]:
history=model.fit(
				X_train,
				Y_train,
				validation_data=(X_test,Y_test),
				batch_size=Batch_Size,
				epochs=Epochs,
				callbacks=[model_checkpoint,reduce_lr]
				)

Epoch 1/100
82/82 [==============================] - 201s 2s/step - loss: 4.0159 - mae: 4.0159 - val_loss: 16.3841 - val_mae: 16.3841 - lr: 0.0010
Epoch 2/100
82/82 [==============================] - 164s 2s/step - loss: 1.3112 - mae: 1.3112 - val_loss: 6.5929 - val_mae: 6.5929 - lr: 0.0010
Epoch 3/100
82/82 [==============================] - 162s 2s/step - loss: 1.0247 - mae: 1.0247 - val_loss: 1.6710 - val_mae: 1.6710 - lr: 0.0010
Epoch 4/100
82/82 [==============================] - 162s 2s/step - loss: 0.7540 - mae: 0.7540 - val_loss: 1.1990 - val_mae: 1.1990 - lr: 0.0010
Epoch 5/100
82/82 [==============================] - 159s 2s/step - loss: 0.7106 - mae: 0.7106 - val_loss: 1.0221 - val_mae: 1.0221 - lr: 0.0010
Epoch 6/100
82/82 [==============================] - 158s 2s/step - loss: 0.7200 - mae: 0.7200 - val_loss: 2.0932 - val_mae: 2.0932 - lr: 0.0010
Epoch 7/100
82/82 [==============================] - 159s 2s/step - loss: 0.6734 - mae: 0.6734 - val_loss: 1.1375 - val_mae: 1.1

In [22]:
model.load_weights(ckp_path)

In [23]:
converter=tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model=converter.convert()

INFO:tensorflow:Assets written to: C:\Users\ishit\AppData\Local\Temp\tmpo02koiss\assets


INFO:tensorflow:Assets written to: C:\Users\ishit\AppData\Local\Temp\tmpo02koiss\assets


In [24]:
with open("model.tflite","wb") as f:
	f.write(tflite_model)

In [25]:
prediction_val=model.predict(X_test,batch_size=32)

15/15 [==============================] - 11s 507ms/step


In [26]:
print(prediction_val[:10])

[[ 0.06613109]
 [15.10586   ]
 [ 9.135483  ]
 [24.054834  ]
 [ 4.073693  ]
 [18.012497  ]
 [ 6.973255  ]
 [ 4.070263  ]
 [10.099316  ]
 [15.113444  ]]


In [27]:
print(Y_test[:10])

[ 0. 15.  9. 24.  4. 18.  7.  4. 10. 15.]
